In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 17 14:11:56 2018

@author: Eric Nost, Environmental Data and Governance Initiative (EDGI)
"""

import requests
import csv
import json
from statistics import *
from collections import Counter
import datetime

In [ ]:
# Prep the data
f = {} # save inspection data by facility name

In [ ]:
# The main script
# Loop through each of the 10 EPA regions and get the ECHO results for compliance information
x=1
while x < 11:
    region=str(x)
    if len(region)<2:
        region = "0"+region
    print("R"+region+"....") # Print out which region we're currently working on
    
    # call up ECHO
    url="https://ofmpub.epa.gov/echo/echo_rest_services.get_facility_info?output=json&p_reg="+region+"&p_act=Y&p_maj=Y&p_med=ALL"
    contents = requests.get(url).content.decode()
    obj = json.loads(contents) # Get the results from ECHO and turn them into something Python can work with
    
    facilities = obj['Results']['Facilities'] # Grab the information on the facilities
    region="R"+region # Name the current region
    
    # Option to break down results by region here...
    
    # For each facility in the EPA region...
    for facility in facilities:
        compliance = facility['Fac3yrComplianceHistory']  # Get its compliance history for the past 12 quarters in the form of "123456789012" #V = violation U = unknown S =sign. violation _ = none
        format_str = '%m/%d/%Y'
        if (compliance[0:3] == "UUU"): #if the first three quarters in the Obama era were Unknown status, skip this facility because they may not have been permitted at all then, only later.
            pass
        else: # these are the facilities we'll work with!
            f[facility['FacName']]={"name":"", "obamaQuarter":"", "thisQuarter":"", "codes":[], "InformalCount":0,"FormalCount":0,"lastInspectionDays":0, "inspectionsFiveYears":0, "InspectionIntervalDeviation":0}
            f[facility['FacName']]["name"] = facility['FacName']
            t = compliance[8:12] # For this analysis, we measure the Trump era as the most recent 4 quarters (Feb 19)
            o = compliance[0:3] # The Obama era is the first 3 quarters of the past 12 (as of Feb 19)
            f[facility['FacName']]['thisQuarter']=compliance[10] # = Q11 on zero index = most recent fully-reported quarter (Feb 19)
            f[facility['FacName']]['obamaQuarter']=compliance[2] # the last Obama quarter in the data (as of Feb 19) 
            for pos, q in enumerate(t):# for each quarter in Trump era, add the compliance code to our list
                f[facility['FacName']]["codes"].append(q) 
            #for pos, q in enumerate(o): 
            try:
                #inspections
                lastInspectionDate = datetime.datetime.strptime(facility["FacDateLastInspection"], format_str) # get the last inspection date. Some facilities have None as a value here - we'll pass them
                lastInspectionDays = int(facility['FacDaysLastInspection']) # how many days (a number) since last inspection
                inspectionsFiveYears=int(facility["FacInspectionCount"])
                
                #analysis: is the most recent inspection interval (days to last inspection) longer than the average inspection interval over the past 5 years?
                avgInspectionsInterval=1825/inspectionsFiveYears #365*5
               
                #collate inspections data by facility
                f[facility['FacName']]["lastInspectionDays"]=lastInspectionDays
                f[facility['FacName']]["inspectionsFiveYears"]=inspectionsFiveYears
                f[facility['FacName']]["InspectionIntervalDeviation"]=lastInspectionDays-avgInspectionsInterval
                      
                #enforcement actions
                InformalCount=int(facility['FacInformalCount'])
                FormalCount=int(facility['FacFormalActionCount'])
                f[facility['FacName']]["InformalCount"]=InformalCount
                f[facility['FacName']]["FormalCount"]=FormalCount
                
            except:
                pass
        # Go to the next facility...
    x+=1 # Go to the next EPA region

In [ ]:
#select only facilities that have had inspections
fac = {k: v for k, v in f.items() if v["inspectionsFiveYears"] > 0} # ~ 200 facilities have no inspections in past 5 years

In [ ]:
#overall compliance rates and differences...
SNC_Obama = (len([v['obamaQuarter'] for k, v in fac.items() if v['obamaQuarter'] == "S"])/len(fac.items()))*100
SNC_Trump = (len([v['thisQuarter'] for k, v in fac.items() if v['thisQuarter'] == "S"])/len(fac.items()))*100

In [ ]:
#calculate medians
medInspectionDays = median([v["lastInspectionDays"] for k,v in fac.items()])
medFormalActions = median([v["FormalCount"] for k,v in fac.items()])
medInformalActions = median([v["InformalCount"] for k,v in fac.items()])

In [ ]:
#select facilities < median and report share by compliance category, in/formal actions
recentlyInspected = {k: v for k, v in fac.items() if v["lastInspectionDays"] < medInspectionDays}

#compliance percents....
SNC_RI=(len([v['thisQuarter'] for k,v in recentlyInspected.items() if v['thisQuarter'] == "S"])/len(recentlyInspected.items()) ) *100
Violation_RI=(len([v['thisQuarter'] for k,v in recentlyInspected.items() if v['thisQuarter'] == "V"])/len(recentlyInspected.items()) ) *100
NV_RI=(len([v['thisQuarter'] for k,v in recentlyInspected.items() if v['thisQuarter'] == "_"])/len(recentlyInspected.items()) ) *100
U_RI=(len([v['thisQuarter'] for k,v in recentlyInspected.items() if v['thisQuarter'] == "U"])/len(recentlyInspected.items()) ) *100

print ("recently inspected facilities = last inspection was within the median last inspection date. In other words, within "+str(medInspectionDays)+" days prior to 2/9/19") 
print("in significant non-compliance: "+str(SNC_RI)+"%")
print("in violation: "+str(Violation_RI)+"%")
print("with no violation: "+str(NV_RI)+"%")
print("unknown status: "+str(U_RI)+"%")

avgFormalActions_RI=mean([v["FormalCount"] for k,v in recentlyInspected.items()])
avgInformalActions_RI=mean([v["InformalCount"] for k,v in recentlyInspected.items()])

In [ ]:
#select facilities > median and report share by compliance category, in/formal actions
notRecentlyInspected = {k: v for k, v in fac.items() if v["lastInspectionDays"] >= medInspectionDays}

SNC_NRI=(len([v['thisQuarter'] for k,v in notRecentlyInspected.items() if v['thisQuarter'] == "S"])/len(notRecentlyInspected.items()) ) *100
Violation_NRI=(len([v['thisQuarter'] for k,v in notRecentlyInspected.items() if v['thisQuarter'] == "V"])/len(notRecentlyInspected.items()) ) *100
NV_NRI=(len([v['thisQuarter'] for k,v in notRecentlyInspected.items() if v['thisQuarter'] == "_"])/len(notRecentlyInspected.items()) ) *100
U_NRI=(len([v['thisQuarter'] for k,v in notRecentlyInspected.items() if v['thisQuarter'] == "U"])/len(notRecentlyInspected.items()) ) *100

print ("NOT recently inspected facilities = last inspection was beyond the median last inspection date. In other words, beyond "+str(medInspectionDays)+" days prior to 2/9/19") 
print("in significant non-compliance: "+str(SNC_NRI)+"%")
print("in violation: "+str(Violation_NRI)+"%")
print("with no violation: "+str(NV_NRI)+"%")
print("unknown status: "+str(U_NRI)+"%")

avgFormalActions_NRI=mean([v["FormalCount"] for k,v in notRecentlyInspected.items()])
avgInformalActions_NRI=mean([v["InformalCount"] for k,v in notRecentlyInspected.items()])

In [ ]:
#report differences between recently and not recently inspected. NOT percent "change"
SNCdiff = (abs(SNC_NRI-SNC_RI)/((SNC_NRI+SNC_RI)/2))*100
Vdiff= (abs(Violation_NRI-Violation_RI)/((Violation_NRI+Violation_RI)/2))*100
NVdiff= (abs(NV_NRI-NV_RI)/((NV_NRI+NV_RI)/2))*100
Udiff= (abs(U_NRI-U_RI)/((U_NRI+U_RI)/2))*100
FAdiff=(abs(avgFormalActions_NRI-avgFormalActions_RI)/((avgFormalActions_NRI+avgFormalActions_RI)/2))*100
IAdiff=(abs(avgInformalActions_NRI-avgInformalActions_RI)/((avgInformalActions_NRI+avgInformalActions_RI)/2))*100

print("percent difference in SNC rates: "+str(SNCdiff)+"%")
print("percent difference in Violation rates: "+str(Vdiff)+"%")
print("percent difference in No Violation rates: "+str(NVdiff)+"%")
print("percent difference in Unknown rates: "+str(Udiff)+"%")
print("percent difference in average # of formal actions: "+str(FAdiff)+"%")
print("percent difference in average # of informal actions: "+str(IAdiff)+"%")

In [ ]:
#report % of facilities with greater than expected inspection interval (>0)
percentWithInspectionsOutOfDate=(len([v["InspectionIntervalDeviation"] for k,v in fac.items() if v["InspectionIntervalDeviation"] > 0]) / len(fac.items()) ) * 100
print("percent of facilities that haven't been inspected in more than their typical interval: "+str(percentWithInspectionsOutOfDate)+"%")

In [ ]:
# OUTPUT: spreadsheet of compliance categories by facility
with open("ECHOscraper.csv", "w", encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    for key, value in f.items():
        try:
            writer.writerow([key, value["codes"][0], value["codes"][1], value["codes"][2], value["codes"][3], value["lastInspectionDays"], value["inspectionsFiveYears"], value["InspectionIntervalDeviation"]])
        except IndexError:
            writer.writerow([key, "ERROR"])
outfile.close()